# Model Training Notebook

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('../notebooks'))))

In [2]:
from utils.utils import setup_mlflow, get_device
from utils.data_loader import load_data
from models.classification.resnet_classification import Resnet18_Classification
from evaluation.classification.eval_classification import plot_confusion_matrix, get_all_preds
from scripts.train_classification import train_classification
import torch
import torchvision.models as models
import mlflow
from sklearn.metrics import classification_report




Classification Training
----------


In [ ]:
data_dir = '../data/coco'
setup_mlflow("classification", "http://localhost:5000") 
device = get_device()
os.makedirs("../evaluation/classification", exist_ok=True)

with mlflow.start_run():
    mlflow.log_param("epochs", 50)
    mlflow.log_param("batch_size", 128)
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("momentum", 0.9)
    
    net = Resnet18_Classification().to(device)
    trainloader, testloader, class_names = load_data(data_dir, batch_size=128, shuffle=True, resize_x=224, resize_y=224)
    trained_net = train_classification(net, trainloader, testloader, device, num_epochs=50)

    save_path = '../models_saved/classification.pth'
    torch.save(trained_net.state_dict(), save_path)
    mlflow.pytorch.log_model(trained_net, "model")
    mlflow.log_artifact(save_path)


    plot_confusion_matrix(trained_net, testloader, class_names, device, save_path="../evaluation/classification/confusion_matrix.png")
    y_pred, y_true = get_all_preds(trained_net, testloader, device)
    class_report = classification_report(y_true, y_pred, target_names=class_names)
    print(class_report)
    
    report_path = "../evaluation/classification/classification_report.txt"
    with open(report_path, "w") as f:
        f.write(class_report)
    mlflow.log_artifact(report_path)

Using GPU: NVIDIA GeForce RTX 4090
Epoch: 1, Mini-Batches: 50, Loss: 0.483, Accuracy: 79.778%
Epoch: 2, Mini-Batches: 50, Loss: 0.341, Accuracy: 83.102%
Epoch: 3, Mini-Batches: 50, Loss: 0.296, Accuracy: 83.934%
Epoch: 4, Mini-Batches: 50, Loss: 0.267, Accuracy: 83.934%
Epoch: 5, Mini-Batches: 50, Loss: 0.232, Accuracy: 83.380%
Epoch: 6, Mini-Batches: 50, Loss: 0.198, Accuracy: 83.380%
Epoch: 7, Mini-Batches: 50, Loss: 0.171, Accuracy: 84.488%


Detection Training
----------


In [ ]:
mlflow.set_experiment("COCO_ResNet_Detection_Training")
with mlflow.start_run(run_name="Detection_Model"):
    train_detection()  

Segmentation Training
----------


In [ ]:
mlflow.set_experiment("COCO_ResNet_Segmentation_Training")
with mlflow.start_run(run_name="Segmentation_Model"):
    train_segmentation()  